## Env set-up

In [14]:
#from google.colab import drive
#drive.mount('/content/drive')

In [15]:
# For emptying trash after each run
#from google.colab import auth
#auth.authenticate_user()
#from googleapiclient.discovery import build
#drive_service = build('drive', 'v3')
#drive_service.files().emptyTrash().execute()

In [16]:
!pwd
!nvidia-smi

/home/nlp-lab-ws23/nlp_praktikum/persuasion_technique_detection/subtask1/transformer/inference
Sat Jan 27 16:53:14 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3080 Ti     On  | 00000000:2B:00.0 Off |                  N/A |
|  0%   35C    P8              25W / 350W |      5MiB / 12288MiB |      0%      Default |
|                                         |                    

In [17]:
#folder_name = "/content/drive/MyDrive/persuasion_technique_detection/"
folder_name = "/home/nlp-lab-ws23/nlp_praktikum/persuasion_technique_detection/"
summary_dir_path = folder_name + "subtask1/transformer/summaries/inference/treshold_randomSearch_1/"

In [18]:
#!pip install transformers datasets wandb evaluate accelerate -qU sklearn_hierarchical_classification sentencepiece

In [19]:
import gc
import re
import json
import numpy as np
import pandas as pd
import random
import torch
import subprocess
import json
import warnings
import shutil
import itertools
import logging
from sklearn import preprocessing
from sklearn.preprocessing import MultiLabelBinarizer,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
from datasets import Dataset,load_dataset,DatasetDict,concatenate_datasets
import datasets
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
import os
from datasets import concatenate_datasets
from transformers import Trainer
from transformers import AutoTokenizer, DataCollatorWithPadding

In [20]:
AVAIL_GPUS = 0
if torch.cuda.is_available():
    device = torch.device("cuda")
    AVAIL_GPUS = torch.cuda.device_count()
    print(f'There are {AVAIL_GPUS} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce RTX 3080 Ti


In [21]:
import wandb
import os

#wandb.login(relogin=True)
wandb.login()

# setup wandb environment variables
os.environ['WANDB_PROJECT'] = "subtask1_transformer_encoder_classification"
os.environ['WANDB_ENTITY'] = "tumnlp"
os.environ["WANDB_LOG_MODEL"]= "end"

## Data Importing and Preprocessing

In [22]:
val_path=folder_name+"data/subtask1/validation.json"
train_path = folder_name+"data/subtask1/train.json"
test_path=folder_name+"data/subtask1/dev_unlabeled.json"

train_files={"train":train_path}
val_files={"val":val_path}
test_files={"test":test_path}

dataset_train=load_dataset("json", data_files=train_files)
dataset_val=load_dataset("json", data_files=val_files)
dataset_test=load_dataset("json",data_files=test_files)

In [23]:
persuasion_unique=[["Ethos","Pathos","Logos"]]
ethos_unique=[["Ad Hominem","Bandwagon","Appeal to authority","Glittering generalities (Virtue)"]]
pathos_unique=[["Exaggeration/Minimisation","Loaded Language","Flag-waving","Appeal to fear/prejudice"]]
logos_unique=[["Justification","Reasoning","Repetition","Obfuscation, Intentional vagueness, Confusion"]]
ad_hominem_unique=[["Name calling/Labeling","Doubt","Smears","Reductio ad hitlerum","Whataboutism"]]
justification_unique=[["Bandwagon","Appeal to authority","Flag-waving","Appeal to fear/prejudice","Slogans"]]
reasoning_unique=[["Distraction","Simplification"]]
distraction_unique=[["Misrepresentation of Someone's Position (Straw Man)","Presenting Irrelevant Data (Red Herring)","Whataboutism"]]
simplification_unique=[["Causal Oversimplification","Black-and-white Fallacy/Dictatorship","Thought-terminating cliché"]]
persuasion_or_not_unique=[["Not Persuasion","Persuasion"]]

mlb_persuasion = MultiLabelBinarizer()
mlb_persuasion.fit(persuasion_unique)

mlb_ethos = MultiLabelBinarizer()
mlb_ethos.fit(ethos_unique)

mlb_pathos = MultiLabelBinarizer()
mlb_pathos.fit(pathos_unique)

mlb_logos = MultiLabelBinarizer()
mlb_logos.fit(logos_unique)

mlb_ad_hominem = MultiLabelBinarizer()
mlb_ad_hominem.fit(ad_hominem_unique)

mlb_justification = MultiLabelBinarizer()
mlb_justification.fit(justification_unique)

mlb_reasoning=MultiLabelBinarizer()
mlb_reasoning.fit(reasoning_unique)

mlb_distraction=MultiLabelBinarizer()
mlb_distraction.fit(distraction_unique)

mlb_simplification=MultiLabelBinarizer()
mlb_simplification.fit(simplification_unique)

lb_persuasion_or_not=LabelEncoder()
lb_persuasion_or_not.fit(persuasion_or_not_unique[0])


LabelEncoder()

In [24]:
def write_json(path,data,test=False):
  if not isinstance(data, dict):
    data = data.to_dict("records")

  with open(path, "w") as output_file:
      json.dump(data, output_file, indent=2,ensure_ascii=False)

## Save multi models in a summary json

In [25]:
api=wandb.Api()
runs=api.runs("subtask1_transformer_encoder_classification")

In [26]:
node_names = ["persuasion_or_not","persuasion","ethos","pathos","logos","ad_hominem","justification","reasoning","simplification","distraction"]
model_nodes = {
    "persuasion_or_not": "model-vinai_bertweet-large-persuasion_or_not-memes_0.5threshold_5e-06learningRate:v3",
    "persuasion": "model-vinai_bertweet-large-persuasion-memes_0.3threshold_5e-06learningRate:v2",
    "ethos": "model-vinai_bertweet-large-ethos-memes_0.4threshold_5e-06learningRate:v2",
    "pathos": "model-vinai_bertweet-large-pathos-memes_0.3threshold_5e-06learningRate:v2",
    "logos": "model-vinai_bertweet-large-logos-memes_0.5threshold_5e-06learningRate:v2",
    "ad_hominem": "model-vinai_bertweet-large-ad_hominem-memes_0.3threshold_5e-06learningRate:v2",
    "justification": "model-vinai_bertweet-large-justification-memes_0.4threshold_5e-06learningRate:v2",
    "reasoning": "model-vinai_bertweet-large-reasoning-memes_0.3threshold_5e-06learningRate:v2",
    "simplification": "model-vinai_bertweet-large-simplification-memes_0.4threshold_5e-06learningRate:v2",
    "distraction": "model-vinai_bertweet-large-distraction-memes_0.5threshold_0.0005learningRate:v2"
  }

threshold_nodes= {
    "persuasion_or_not": 0.5,
    "persuasion": 0.3,
    "ethos": 0.4,
    "pathos": 0.3,
    "logos": 0.5,
    "ad_hominem": 0.3,
    "justification": 0.4,
    "reasoning": 0.3,
    "simplification": 0.4,
    "distraction": 0.5
}
checkpoints= {
    "persuasion_or_not": "vinai/bertweet-large",
    "persuasion": "vinai/bertweet-large",
    "ethos": "vinai/bertweet-large",
    "pathos": "vinai/bertweet-large",
    "logos": "vinai/bertweet-large",
    "ad_hominem": "vinai/bertweet-large",
    "justification": "vinai/bertweet-large",
    "reasoning": "vinai/bertweet-large",
    "simplification": "vinai/bertweet-large",
    "distraction": "vinai/bertweet-large"
}

best_multiple_models={"model_nodes": model_nodes,"threshold_nodes":threshold_nodes,"checkpoints":checkpoints}
best_multiple_models

{'model_nodes': {'persuasion_or_not': 'model-vinai_bertweet-large-persuasion_or_not-memes_0.5threshold_5e-06learningRate:v3',
  'persuasion': 'model-vinai_bertweet-large-persuasion-memes_0.3threshold_5e-06learningRate:v2',
  'ethos': 'model-vinai_bertweet-large-ethos-memes_0.4threshold_5e-06learningRate:v2',
  'pathos': 'model-vinai_bertweet-large-pathos-memes_0.3threshold_5e-06learningRate:v2',
  'logos': 'model-vinai_bertweet-large-logos-memes_0.5threshold_5e-06learningRate:v2',
  'ad_hominem': 'model-vinai_bertweet-large-ad_hominem-memes_0.3threshold_5e-06learningRate:v2',
  'justification': 'model-vinai_bertweet-large-justification-memes_0.4threshold_5e-06learningRate:v2',
  'reasoning': 'model-vinai_bertweet-large-reasoning-memes_0.3threshold_5e-06learningRate:v2',
  'simplification': 'model-vinai_bertweet-large-simplification-memes_0.4threshold_5e-06learningRate:v2',
  'distraction': 'model-vinai_bertweet-large-distraction-memes_0.5threshold_0.0005learningRate:v2'},
 'threshold_n

## Multi model evaluation

In [27]:
def tokenize(examples,max_length=None):

  if max_length is not None:

    encoding = tokenizer(examples["text"], truncation=True,padding=True,max_length=max_length)
  else:
    encoding = tokenizer(examples["text"], truncation=True,padding=True)
  return encoding

def get_preds(binarizer,predicted_logits,threshold=0.5):

  predicted_logits_tensor = torch.from_numpy(predicted_logits)

  if isinstance(binarizer,LabelEncoder):
    softmax=torch.nn.Softmax(dim=1)
    full_probs=softmax(predicted_logits_tensor)

    probs,predictions=torch.max(full_probs,dim=1)
    predictions=[[p] for p in binarizer.inverse_transform(predictions)]
    probs = [[value.item()] for value in probs]
    res=[(x, y) for x, y in zip(predictions, probs)]
    return res
  else:
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(predicted_logits_tensor)
    preds=(probs>threshold).int()


    mask = (preds == 1)

    # Use the mask to select the corresponding probabilities
    selected_probs_list = []

    # Iterate over rows and select probabilities using the mask
    for i in range(preds.size(0)):
        selected_probs_row = probs[i][mask[i]].tolist()
        selected_probs_list.append(selected_probs_row)

    names=binarizer.inverse_transform(preds)
    res=[(list(x), y) for x, y in zip(names, selected_probs_list)]
    return res

project_name="subtask1_transformer_encoder_classification"

def return_trainer(model_name,unique_labels):
  api = wandb.Api()
  artifact=api.artifact(model_name)
  
  model_dir = folder_name + "subtask1/transformer/inference/artifacts/" + model_name
  if not os.path.exists(model_dir):
    model_dir = artifact.download()  
  
  if "persuasion_or_not" in model_name:
    lbls=unique_labels[0]
    id2label={0:lbls[0],1:lbls[1]}
    label2id={lbls[0]:0,lbls[1]:1}
    model = AutoModelForSequenceClassification.from_pretrained(model_dir, num_labels=len(unique_labels[0]),id2label=id2label, label2id=label2id)
  else:
    model = AutoModelForSequenceClassification.from_pretrained(model_dir, num_labels=len(unique_labels[0]), problem_type="multi_label_classification", ignore_mismatched_sizes=True)

  trainer = Trainer(model=model)
  if torch.cuda.is_available():
    trainer.model = model.cuda()
  return trainer



### Val set eval

In [28]:
prediction_set=dataset_val["val"]
prediction_set=prediction_set.remove_columns(["labels"])

In [29]:
def get_new_labels(model_address,node,unique,threshold,binarizer):
  trainer=return_trainer(model_address,unique)
  preds=get_preds(binarizer,trainer.predict(curr_set).predictions,threshold)

  for i in range(len(preds)):

    if len(preds[i][0])>0:
      t=prev_labels[i][0].index(node)
      _=prev_labels[i][1].pop(t)
      prev_labels[i][0].remove(node)
      parent_labels=prev_labels[i][0]
      parent_probs=prev_labels[i][1]

      new_labels.append((parent_labels+preds[i][0],parent_probs+preds[i][1]))
    else:
      new_labels.append(prev_labels[i])

In [30]:
def hierarchical_scores(gold_label_path, pred_label_path):
    scorer = folder_name + "subtask1/subtask_1_2a.py"
    command = f'python3 {scorer} --gold_file_path {gold_label_path} --pred_file_path {pred_label_path}'

    result = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, text=True)
    output = result.stdout.strip()

    parts = output.split('\t')
    f1_h = parts[0].split('=')[1]
    prec_h = parts[1].split('=')[1]
    rec_h = parts[2].split('=')[1]

    f1_h = float(f1_h)
    prec_h = float(prec_h)
    rec_h = float(rec_h)

    hierarchical_metrics = {"f1_hierarchical": f1_h, "precision_hierarchical": prec_h, "recall_hierarchical": rec_h}
    return hierarchical_metrics

In [31]:
trainer=return_trainer(best_multiple_models["model_nodes"]["persuasion_or_not"],persuasion_or_not_unique)
threshold=best_multiple_models["threshold_nodes"]['persuasion_or_not']
unique_models=list(set(best_multiple_models["checkpoints"].values()))
dict_prediction_set_tokenized=dict()

for m in unique_models:
  tokenizer = AutoTokenizer.from_pretrained(m)
  if m=="vinai/bertweet-large" or m=="microsoft/deberta-v3-base":

    prediction_set_tokenized=prediction_set.map(tokenize ,fn_kwargs={"max_length":512},batched=True)
  else:

    prediction_set_tokenized=prediction_set.map(tokenize ,batched=True)

  dict_prediction_set_tokenized[m]=prediction_set_tokenized

dataset=dict_prediction_set_tokenized[best_multiple_models["checkpoints"]["persuasion_or_not"]]
preds_=get_preds(lb_persuasion_or_not,trainer.predict(dataset).predictions,threshold)
preds=[el if el[0][0]=="Persuasion" else [] for el in preds_]
final_ds=dict(zip(dataset["id"],preds))

model_seq=["Persuasion","Ethos","Pathos","Logos","Ad Hominem","Justification","Reasoning","Simplification","Distraction"]
node_names = ["persuasion","ethos","pathos","logos","ad_hominem","justification","reasoning","simplification","distraction"]
unique_node_labels = [persuasion_unique, ethos_unique, pathos_unique, logos_unique, ad_hominem_unique, justification_unique, reasoning_unique, distraction_unique, simplification_unique]
mlb_nodes = [mlb_persuasion, mlb_ethos, mlb_pathos, mlb_logos, mlb_ad_hominem, mlb_justification, mlb_reasoning, mlb_simplification, mlb_distraction]

wandb: Downloading large artifact model-vinai_bertweet-large-persuasion_or_not-memes_0.5threshold_5e-06learningRate:v3, 1358.86MB. 8 files... 
wandb:   8 of 8 files downloaded.  
Done. 0:0:0.8


In [32]:
for i,node in enumerate(model_seq):

  ids=[id for id,val in final_ds.items() if (len(val)>0) and (len(val[0])>0) and (node in val[0])]

  prev_labels=[final_ds[x] for x in ids]
  new_labels=[]
  dataset=dict_prediction_set_tokenized[best_multiple_models["checkpoints"][node_names[i]]]
  curr_set=dataset.filter(lambda x: x["id"] in ids)

  print(f"Node: {node}, curr_set: {curr_set}")
  if len(ids) == 0:
    warnings.warn(f"No predictions with Label {node} made!!!!!")
    continue

  tokenizer = AutoTokenizer.from_pretrained(best_multiple_models["checkpoints"][node_names[i]])
  get_new_labels(best_multiple_models["model_nodes"][node_names[i]],node,unique_node_labels[i],best_multiple_models["threshold_nodes"][node_names[i]],mlb_nodes[i])

  for k in range(len(ids)):
    final_ds[ids[k]]=new_labels[k]
  print(f"final_ds of {node}:")
  print(final_ds)

top_k=9
final_df=dict()
for k,v in final_ds.items():
  if len(v)>0:
    pred_prob=list(zip(v[0],v[1]))
    pred_prob.sort(key=lambda x: -x[1])

    tmp=[]
    i=0
    for el in pred_prob:
      if i==top_k:
        break
      if el[0] not in tmp:
        tmp.append(el[0])
        i+=1
    final_df[str(k)]=tmp
  else:
    final_df[str(k)]=v

pred_df=pd.DataFrame.from_dict({"id":final_df.keys(),"labels":final_df.values()})

val_pred_file=summary_dir_path + "val_pred.json"
if not os.path.exists(summary_dir_path):
  os.makedirs(summary_dir_path)

write_json(val_pred_file,pred_df)

Node: Persuasion, curr_set: Dataset({
    features: ['id', 'link', 'text', 'input_ids', 'attention_mask'],
    num_rows: 427
})


wandb: Downloading large artifact model-vinai_bertweet-large-persuasion-memes_0.3threshold_5e-06learningRate:v2, 1358.86MB. 8 files... 
wandb:   8 of 8 files downloaded.  
Done. 0:0:0.7


final_ds of Persuasion:
{'63135': (['Ethos', 'Logos'], [0.7346245646476746, 0.9095005989074707]), '78590': [], '65834': [], '67394': (['Ethos'], [0.8869884610176086]), '67709': (['Ethos', 'Logos'], [0.9864329695701599, 0.9499585628509521]), '66205': (['Ethos', 'Logos', 'Pathos'], [0.9531282186508179, 0.9699998497962952, 0.9701639413833618]), '79029': (['Ethos', 'Pathos'], [0.477974534034729, 0.8078124523162842]), '78559': (['Ethos', 'Logos', 'Pathos'], [0.6477234959602356, 0.9482312202453613, 0.8120328783988953]), '77741': (['Ethos', 'Pathos'], [0.9036175608634949, 0.5642415285110474]), '63599': (['Ethos', 'Logos'], [0.9815773367881775, 0.37886112928390503]), '77552': (['Ethos', 'Logos', 'Pathos'], [0.9863197207450867, 0.9785575270652771, 0.935269832611084]), '71273': (['Logos', 'Pathos'], [0.9266861081123352, 0.9630942940711975]), '65211': (['Ethos', 'Logos'], [0.9897456169128418, 0.3122675120830536]), '71723': (['Ethos', 'Logos', 'Pathos'], [0.9912900924682617, 0.6566562056541443, 0.

wandb: Downloading large artifact model-vinai_bertweet-large-ethos-memes_0.4threshold_5e-06learningRate:v2, 1358.87MB. 8 files... 
wandb:   8 of 8 files downloaded.  
Done. 0:0:0.7


final_ds of Ethos:
{'63135': (['Logos', 'Ad Hominem'], [0.9095005989074707, 0.9989981055259705]), '78590': [], '65834': [], '67394': (['Glittering generalities (Virtue)'], [0.9960200190544128]), '67709': (['Logos', 'Ad Hominem'], [0.9499585628509521, 0.9993318915367126]), '66205': (['Logos', 'Pathos', 'Appeal to authority'], [0.9699998497962952, 0.9701639413833618, 0.9980031847953796]), '79029': (['Pathos', 'Ad Hominem'], [0.8078124523162842, 0.998624324798584]), '78559': (['Logos', 'Pathos', 'Ad Hominem', 'Appeal to authority'], [0.9482312202453613, 0.8120328783988953, 0.9698143601417542, 0.9980880618095398]), '77741': (['Pathos', 'Glittering generalities (Virtue)'], [0.5642415285110474, 0.9964690208435059]), '63599': (['Logos', 'Ad Hominem', 'Appeal to authority'], [0.37886112928390503, 0.9994444251060486, 0.5861703753471375]), '77552': (['Logos', 'Pathos', 'Ad Hominem', 'Appeal to authority'], [0.9785575270652771, 0.935269832611084, 0.997986912727356, 0.9979543685913086]), '71273': 

...

final_ds of Reasoning:
{'63135': (['Smears', 'Simplification'], [0.9625642895698547, 0.9980992674827576]), '78590': [], '65834': [], '67394': (['Glittering generalities (Virtue)'], [0.9960200190544128]), '67709': (['Smears', 'Appeal to authority'], [0.9864019751548767, 0.992642343044281]), '66205': (['Appeal to authority', 'Flag-waving', 'Loaded Language', 'Appeal to authority'], [0.9980031847953796, 0.7430326342582703, 0.8994461297988892, 0.995482325553894]), '79029': (['Flag-waving', 'Loaded Language', 'Smears'], [0.31907618045806885, 0.9201692342758179, 0.9560523629188538]), '78559': (['Appeal to authority', 'Appeal to fear/prejudice', 'Loaded Language', 'Smears', 'Simplification'], [0.9980880618095398, 0.5396066308021545, 0.7668426632881165, 0.9794070720672607, 0.9980744123458862]), '77741': (['Glittering generalities (Virtue)', 'Exaggeration/Minimisation', 'Flag-waving'], [0.9964690208435059, 0.49960121512413025, 0.7555371522903442]), '63599': (['Appeal to authority', 'Name callin

wandb: Downloading large artifact model-vinai_bertweet-large-simplification-memes_0.4threshold_5e-06learningRate:v2, 1358.86MB. 8 files... 
wandb:   8 of 8 files downloaded.  
Done. 0:0:0.8


final_ds of Simplification:
{'63135': (['Smears', 'Black-and-white Fallacy/Dictatorship'], [0.9625642895698547, 0.9989990592002869]), '78590': [], '65834': [], '67394': (['Glittering generalities (Virtue)'], [0.9960200190544128]), '67709': (['Smears', 'Appeal to authority'], [0.9864019751548767, 0.992642343044281]), '66205': (['Appeal to authority', 'Flag-waving', 'Loaded Language', 'Appeal to authority'], [0.9980031847953796, 0.7430326342582703, 0.8994461297988892, 0.995482325553894]), '79029': (['Flag-waving', 'Loaded Language', 'Smears'], [0.31907618045806885, 0.9201692342758179, 0.9560523629188538]), '78559': (['Appeal to authority', 'Appeal to fear/prejudice', 'Loaded Language', 'Smears', 'Black-and-white Fallacy/Dictatorship'], [0.9980880618095398, 0.5396066308021545, 0.7668426632881165, 0.9794070720672607, 0.9990621209144592]), '77741': (['Glittering generalities (Virtue)', 'Exaggeration/Minimisation', 'Flag-waving'], [0.9964690208435059, 0.49960121512413025, 0.7555371522903442]

wandb: Downloading large artifact model-vinai_bertweet-large-distraction-memes_0.5threshold_0.0005learningRate:v2, 1358.86MB. 8 files... 
wandb:   8 of 8 files downloaded.  
Done. 0:0:0.7


final_ds of Distraction:
{'63135': (['Smears', 'Black-and-white Fallacy/Dictatorship'], [0.9625642895698547, 0.9989990592002869]), '78590': [], '65834': [], '67394': (['Glittering generalities (Virtue)'], [0.9960200190544128]), '67709': (['Smears', 'Appeal to authority'], [0.9864019751548767, 0.992642343044281]), '66205': (['Appeal to authority', 'Flag-waving', 'Loaded Language', 'Appeal to authority'], [0.9980031847953796, 0.7430326342582703, 0.8994461297988892, 0.995482325553894]), '79029': (['Flag-waving', 'Loaded Language', 'Smears'], [0.31907618045806885, 0.9201692342758179, 0.9560523629188538]), '78559': (['Appeal to authority', 'Appeal to fear/prejudice', 'Loaded Language', 'Smears', 'Black-and-white Fallacy/Dictatorship'], [0.9980880618095398, 0.5396066308021545, 0.7668426632881165, 0.9794070720672607, 0.9990621209144592]), '77741': (['Glittering generalities (Virtue)', 'Exaggeration/Minimisation', 'Flag-waving'], [0.9964690208435059, 0.49960121512413025, 0.7555371522903442]), 

In [33]:
hierarchical_metrics = hierarchical_scores(val_path, val_pred_file)
hierarchical_metrics

{'f1_hierarchical': 0.67663,
 'precision_hierarchical': 0.61346,
 'recall_hierarchical': 0.75431}

In [ ]:
lvl1_parents={"Appeal to authority":["Ethos","Logos"],
              "Appeal to fear/prejudice":["Pathos","Logos"],
              "Bandwagon":["Ethos","Logos"],
              "Black-and-white Fallacy/Dictatorship":["Logos"],
              "Causal Oversimplification":["Logos"],
              "Doubt":["Ethos"],
              "Exaggeration/Minimisation":["Pathos"],
              "Flag-waving":["Pathos","Logos"],
              "Glittering generalities (Virtue)":["Ethos"],
              "Loaded Language":["Pathos"],
              "Misrepresentation of Someone's Position (Straw Man)":["Logos"],
              "Name calling/Labeling":["Ethos"],
              "Obfuscation, Intentional vagueness, Confusion":["Logos"],
              "Presenting Irrelevant Data (Red Herring)":["Logos"],
              "Reductio ad hitlerum":["Ethos"],
              "Repetition":["Logos"],
              "Slogans":["Logos"],
              "Smears":["Ethos"],
              "Thought-terminating cliché":["Logos"],
              "Whataboutism":["Ethos","Logos"],
              "Persuasion": [],
              "Ethos": ['Persuasion'],
              "Pathos": ['Persuasion'],
              "Logos": ['Persuasion'],
              "Ad Hominem": ['Ethos'],
              "Justification": ['Logos'],
              "Reasoning": ['Reasoning'],
              "Simplification": ['Reasoning'],
              "Distraction": ['Reasoning'],}

In [ ]:
def get_preds(binarizer,predicted_logits,threshold=0.5):

  predicted_logits_tensor = torch.from_numpy(predicted_logits)

  if isinstance(binarizer,LabelEncoder):
    softmax=torch.nn.Softmax(dim=1)
    full_probs=softmax(predicted_logits_tensor)

    probs,predictions=torch.max(full_probs,dim=1)
    predictions=[[p] for p in binarizer.inverse_transform(predictions)]
    probs = [[value.item()] for value in probs]
    res=[(x, y) for x, y in zip(predictions, probs)]
    return res
  else:
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(predicted_logits_tensor)
    preds=(probs>threshold).int()


    mask = (preds == 1)

    # Use the mask to select the corresponding probabilities
    selected_probs_list = []

    # Iterate over rows and select probabilities using the mask
    for i in range(preds.size(0)):
        selected_probs_row = probs[i][mask[i]].tolist()
        selected_probs_list.append(selected_probs_row)

    names=binarizer.inverse_transform(preds)
    res=[(list(x), y) for x, y in zip(names, selected_probs_list)]
    return res

In [ ]:
trainer=return_trainer(best_multiple_models["model_nodes"]["persuasion_or_not"],persuasion_or_not_unique)
threshold=best_multiple_models["threshold_nodes"]['persuasion_or_not']
dataset=dict_prediction_set_tokenized[best_multiple_models["checkpoints"]["persuasion_or_not"]]
preds_=get_preds(lb_persuasion_or_not,trainer.predict(dataset).predictions,threshold)
preds=[el if el[0][0]=="Persuasion" else [] for el in preds_]
final_ds=dict(zip(dataset["id"],preds))

for i,node in enumerate(model_seq):

  ids=[id for id,val in final_ds.items() if (len(val)>0) and (len(val[0])>0) and (node in val[0])]

  prev_labels=[final_ds[x] for x in ids]
  new_labels=[]
  dataset=dict_prediction_set_tokenized[best_multiple_models["checkpoints"][node_names[i]]]
  curr_set=dataset.filter(lambda x: x["id"] in ids)

  print(f"Node: {node}, curr_set: {curr_set}")
  if len(ids) == 0:
    warnings.warn(f"No predictions with Label {node} made!!!!!")
    continue

  tokenizer = AutoTokenizer.from_pretrained(best_multiple_models["checkpoints"][node_names[i]])
  get_new_labels(best_multiple_models["model_nodes"][node_names[i]],node,unique_node_labels[i],best_multiple_models["threshold_nodes"][node_names[i]],mlb_nodes[i])

  for k in range(len(ids)):
    final_ds[ids[k]]=new_labels[k]
  print("final_ds:")
  print(final_ds)

top_k=9
final_df=dict()
for k,v in final_ds.items():
  if len(v)>0:
    pred_prob=list(zip(v[0],v[1]))
    pred_prob.sort(key=lambda x: -x[1])

    tmp=[]
    i=0
    for el in pred_prob:
      if i==top_k:
        break
      if el[0] not in tmp:
        tmp.append(el[0])
        i+=1
    final_df[str(k)]=tmp
  else:
    final_df[str(k)]=v

reversed_pred_df=pd.DataFrame.from_dict({"id":final_df.keys(),"labels":final_df.values()})

reversed_val_pred_file=summary_dir_path + "reversed_val_pred.json"
if not os.path.exists(summary_dir_path):
  os.makedirs(summary_dir_path)

write_json(reversed_val_pred_file,reversed_pred_df)

In [ ]:
hierarchical_metrics = hierarchical_scores(val_path, reversed_val_pred_file)
hierarchical_metrics

In [22]:
best_multiple_models["top_k"] = top_k
best_multiple_models["Validation score"] = hierarchical_metrics
best_multiple_models_file_path=summary_dir_path+"summary.json"
write_json(best_multiple_models_file_path, best_multiple_models)

## Dev set eval

In [26]:
prediction_set=dataset_test["test"]

In [27]:
trainer=return_trainer(best_multiple_models["model_nodes"]["persuasion_or_not"],persuasion_or_not_unique)
threshold=best_multiple_models["threshold_nodes"]['persuasion_or_not']
unique_models=list(set(best_multiple_models["checkpoints"].values()))
dict_prediction_set_tokenized=dict()

for m in unique_models:
  tokenizer = AutoTokenizer.from_pretrained(m)
  if m=="vinai/bertweet-large" or m=="microsoft/deberta-v3-base":

    prediction_set_tokenized=prediction_set.map(tokenize ,fn_kwargs={"max_length":512},batched=True)
  else:

    prediction_set_tokenized=prediction_set.map(tokenize ,batched=True)

  dict_prediction_set_tokenized[m]=prediction_set_tokenized

dataset=dict_prediction_set_tokenized[best_multiple_models["checkpoints"]["persuasion_or_not"]]
preds_=get_preds(lb_persuasion_or_not,trainer.predict(dataset).predictions,threshold)
preds=[el if el[0][0]=="Persuasion" else [] for el in preds_]
final_ds=dict(zip(dataset["id"],preds))

model_seq=["Persuasion","Ethos","Pathos","Logos","Ad Hominem","Justification","Reasoning","Simplification","Distraction"]
node_names = ["persuasion","ethos","pathos","logos","ad_hominem","justification","reasoning","simplification","distraction"]
unique_node_labels = [persuasion_unique, ethos_unique, pathos_unique, logos_unique, ad_hominem_unique, justification_unique, reasoning_unique, distraction_unique, simplification_unique]
mlb_nodes = [mlb_persuasion, mlb_ethos, mlb_pathos, mlb_logos, mlb_ad_hominem, mlb_justification, mlb_reasoning, mlb_simplification, mlb_distraction]

for i,node in enumerate(model_seq):

  ids=[id for id,val in final_ds.items() if (len(val)>0) and (len(val[0])>0) and (node in val[0])]

  prev_labels=[final_ds[x] for x in ids]
  new_labels=[]
  dataset=dict_prediction_set_tokenized[best_multiple_models["checkpoints"][node_names[i]]]
  curr_set=dataset.filter(lambda x: x["id"] in ids)

  print(f"Node: {node}, curr_set: {curr_set}")
  if len(ids) == 0:
    warnings.warn(f"No predictions with Label {node} made!!!!!")
    continue

  tokenizer = AutoTokenizer.from_pretrained(best_multiple_models["checkpoints"][node_names[i]])
  get_new_labels(best_multiple_models["model_nodes"][node_names[i]],node,unique_node_labels[i],best_multiple_models["threshold_nodes"][node_names[i]],mlb_nodes[i])

  for k in range(len(ids)):
    final_ds[ids[k]]=new_labels[k]

top_k=3
final_df=dict()
for k,v in final_ds.items():
  if len(v)>0:
    pred_prob=list(zip(v[0],v[1]))
    pred_prob.sort(key=lambda x: -x[1])

    tmp=[]
    i=0
    for el in pred_prob:
      if i==top_k:
        break
      if el[0] not in tmp:
        tmp.append(el[0])
        i+=1
    final_df[str(k)]=tmp
  else:
    final_df[str(k)]=v

pred_df=pd.DataFrame.from_dict({"id":final_df.keys(),"labels":final_df.values()})

dev_pred_file=summary_dir_path + "dev_pred.json"
if not os.path.exists(summary_dir_path):
  os.makedirs(summary_dir_path)

write_json(dev_pred_file,pred_df)

Node: Persuasion, curr_set: Dataset({
    features: ['text', 'id', 'input_ids', 'attention_mask'],
    num_rows: 870
})


Node: Ethos, curr_set: Dataset({
    features: ['text', 'id', 'input_ids', 'attention_mask'],
    num_rows: 676
})


Node: Pathos, curr_set: Dataset({
    features: ['text', 'id', 'input_ids', 'attention_mask'],
    num_rows: 508
})


Node: Logos, curr_set: Dataset({
    features: ['text', 'id', 'input_ids', 'attention_mask'],
    num_rows: 641
})


Node: Ad Hominem, curr_set: Dataset({
    features: ['text', 'id', 'input_ids', 'attention_mask'],
    num_rows: 551
})


Node: Justification, curr_set: Dataset({
    features: ['text', 'id', 'input_ids', 'attention_mask'],
    num_rows: 366
})


Node: Reasoning, curr_set: Dataset({
    features: ['text', 'id', 'input_ids', 'attention_mask'],
    num_rows: 364
})


Node: Simplification, curr_set: Dataset({
    features: ['text', 'id', 'input_ids', 'attention_mask'],
    num_rows: 310
})


Node: Distraction, curr_set: Dataset({
    features: ['text', 'id', 'input_ids', 'attention_mask'],
    num_rows: 66
})


In [28]:
def delete_dir(path):
    if os.path.exists(path):
        try:
            shutil.rmtree(path)
            print(f"The directory {path} and all its contents have been deleted successfully")
        except OSError as e:
            print(f"Error: {e.strerror}")
    else:
        print(f"The directory {path} does not exist")

In [29]:
path = folder_name + "subtask1"
path = path + "/transformer/inference/"
delete_dir(path + "tumnlp")
delete_dir(path + "wandb")
delete_dir(path + "artifacts")
delete_dir(path + "tmp_trainer")

The directory /home/nlp-lab-ws23/nlp_praktikum/persuasion_technique_detection/subtask1/transformer/inference/tumnlp does not exist
The directory /home/nlp-lab-ws23/nlp_praktikum/persuasion_technique_detection/subtask1/transformer/inference/wandb does not exist
The directory /home/nlp-lab-ws23/nlp_praktikum/persuasion_technique_detection/subtask1/transformer/inference/artifacts and all its contents have been deleted successfully
The directory /home/nlp-lab-ws23/nlp_praktikum/persuasion_technique_detection/subtask1/transformer/inference/tmp_trainer and all its contents have been deleted successfully
